In [90]:
import pandas as pd
import numpy as np
import sqlite3

In [91]:
#creating SQL connection
conn = sqlite3.connect('../Data/pet_care_industry.db')
c = conn.cursor()

#function to close connection
def close_c_conn():
    c.close()
    conn.close()

In [98]:
#initialize dog growth table with unique list of zipcodes
dog_growth = pd.read_sql_query('SELECT DISTINCT Borough FROM dogs', conn)
print(dog_growth.shape)
dog_growth.head()

(79, 1)


,Borough
0,Queens
1,Brooklyn
2,QUEENS
3,Manhattan
4,Bronx


In [99]:
#function to select dogs by zipcodes for a given year
def get_dogs_by_zipcode_in_year(year):
    return pd.read_sql_query("""SELECT Borough, COUNT(Borough) AS CY{}
                                FROM dogs
                                GROUP BY Borough
                                HAVING LicenseIssuedYear <= {};""".format(year, year), conn)

#merge dogs by zipcodes for all active years
for i in range(2014, 2023):
    dog_growth = pd.merge(dog_growth, get_dogs_by_zipcode_in_year(i), on='Borough', how='left')
    
#dog_growth = dog_growth.fillna(0).astype(int)
dog_growth

,Borough,CY2014,CY2015,CY2016,CY2017,CY2018,CY2019,CY2020,CY2021,CY2022
0,Queens,24658.0,24658.0,24658,24658,24658,24658,24658,24658,24658
1,Brooklyn,NaN,30036.0,30036,30036,30036,30036,30036,30036,30036
2,QUEENS,NaN,320.0,320,320,320,320,320,320,320
3,Manhattan,NaN,NaN,42344,42344,42344,42344,42344,42344,42344
4,Bronx,NaN,NaN,12485,12485,12485,12485,12485,12485,12485
5,South Richmond Hil,NaN,NaN,1,1,1,1,1,1,1
6,SO RICHMOND,NaN,NaN,1,1,1,1,1,1,1
7,Richmond Hill,NaN,NaN,2,2,2,2,2,2,2
8,College Point,NaN,1.0,1,1,1,1,1,1,1
9,Quens,NaN,1.0,1,1,1,1,1,1,1


In [97]:
dog_growth['cagr'] = (dog_growth['CY2016'] / dog_growth['CY2014']) ** (1/2) - 1
dog_growth = dog_growth.sort_values(by=['cagr'], ascending=False)
dog_growth = dog_growth.replace(np.inf, np.nan)
dog_growth = dog_growth.dropna(subset=['cagr'])
dog_growth

,BreedName,CY2014,CY2015,CY2016,CY2017,CY2018,CY2019,CY2020,CY2021,CY2022,cagr
0,Unknown,16763.0,16763.0,16763,16763,16763,16763,16763,16763,16763,0.0
1,Labrador Retriever Crossbreed,2773.0,2773.0,2773,2773,2773,2773,2773,2773,2773,0.0
2,American Pit Bull Terrier/Pit Bull,3341.0,3341.0,3341,3341,3341,3341,3341,3341,3341,0.0
3,Labrador Retriever,4195.0,4195.0,4195,4195,4195,4195,4195,4195,4195,0.0
4,Shih Tzu,7133.0,7133.0,7133,7133,7133,7133,7133,7133,7133,0.0
5,Boxer,1207.0,1207.0,1207,1207,1207,1207,1207,1207,1207,0.0
6,"Schnauzer, Miniature",1139.0,1139.0,1139,1139,1139,1139,1139,1139,1139,0.0
7,Yorkshire Terrier,7747.0,7747.0,7747,7747,7747,7747,7747,7747,7747,0.0
126,"Dachshund, Wirehaired, Miniature",69.0,69.0,69,69,69,69,69,69,69,0.0
127,"Schnauzer, Giant",31.0,31.0,31,31,31,31,31,31,31,0.0


In [ ]:
#total share = total number of dogs

In [ ]:
#close connection
close_c_conn()